<a href="https://colab.research.google.com/github/Angelin5/skills-introduction-to-github/blob/main/harvard_artifacts_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 1. Kill old processes ---
!pkill -f streamlit
!pkill -f cloudflared

# --- 2. Save Harvard Artifacts app ---
app_code = """
import streamlit as st
import requests
import pandas as pd
import sqlite3
import time

st.set_page_config(page_title="Harvard Artifacts", page_icon="🏛", layout="wide")

# --- Title (centered) ---
st.markdown("<h2 style='text-align:center; font-weight:bold;'>🎨 Harvard Artifacts Dashboard</h2>", unsafe_allow_html=True)

API_KEY = "bf7eb21e-2877-4aec-8f10-c14c65469b33"
URL = "https://api.harvardartmuseums.org/object"

if "records" not in st.session_state:
    st.session_state.records = []
if "step" not in st.session_state:
    st.session_state.step = "start"
if "db_ready" not in st.session_state:
    st.session_state.db_ready = False

# --- Step 1: Select Classification ---
classification = st.selectbox(
    "Select Classification",
    ["Coins", "Sculpture", "Drawings", "Prints", "Photographs"]
)

# --- Collect Data ---
if st.button("Collect Data"):
    all_records = []
    page = 1
    size = 100  # max allowed by API
    target = 2500

    st.info(f"⏳ Collecting {target} {classification} records... please wait")

    while len(all_records) < target:
        params = {
            "classification": classification,
            "size": size,
            "page": page,
            "apikey": API_KEY
        }
        response = requests.get(URL, params=params)

        if response.status_code != 200:
            st.error("❌ Failed to fetch data")
            break

        data = response.json().get("records", [])
        if not data:
            break  # no more records available

        all_records.extend(data)
        page += 1
        time.sleep(0.2)  # avoid rate-limit

    st.session_state.records = all_records[:target]
    st.session_state.step = "actions"
    st.success(f"✅ Collected {len(st.session_state.records)} {classification} records")

# --- Step 2: Action buttons ---
if st.session_state.step in ["actions","show_choice","migrate","sql"] and st.session_state.records:
    c1, c2, c3 = st.columns(3)
    with c1:
        if st.button("Select Your Choice"):
            st.session_state.step = "show_choice"
    with c2:
        if st.button("Migrate to SQL"):
            st.session_state.step = "migrate"
    with c3:
        if st.button("SQL Queries"):
            st.session_state.step = "sql"

# --- Step 3a: Show Metadata / Media / Colors ---
if st.session_state.step == "show_choice":
    meta_col, media_col, color_col = st.columns(3)

    with meta_col:
        st.subheader("Metadata")
        for rec in st.session_state.records[:50]:  # preview only 50
            st.json({
                "id": rec.get("id"),
                "title": rec.get("title"),
                "culture": rec.get("culture"),
                "period": rec.get("period"),
                "century": rec.get("century"),
                "medium": rec.get("medium"),
                "dimensions": rec.get("dimensions"),
                "description": rec.get("description"),
                "department": rec.get("department"),
                "classification": rec.get("classification"),
                "accessionyear": rec.get("accessionyear"),
                "accessionmethod": rec.get("accessionmethod")
            })

    with media_col:
        st.subheader("Media")
        for rec in st.session_state.records[:50]:
            st.json({
                "objectid": rec.get("objectid"),
                "imagecount": rec.get("imagecount"),
                "mediacount": rec.get("mediacount"),
                "colorcount": rec.get("colorcount"),
                "rank": rec.get("rank"),
                "datebegin": rec.get("datebegin"),
                "dateend": rec.get("dateend")
            })

    with color_col:
        st.subheader("Colors")
        for rec in st.session_state.records[:50]:
            if rec.get("colors"):
                for j in rec["colors"]:
                    st.json({
                        "objectid": rec.get("objectid"),
                        "color": j.get("color"),
                        "spectrum": j.get("spectrum"),
                        "hue": j.get("hue"),
                        "percent": j.get("percent"),
                        "css3": j.get("css3")
                    })

# --- Step 3b: Migrate to SQL ---
if st.session_state.step == "migrate":
    st.markdown("### Insert the collected data")
    if st.button("Insert"):
        conn = sqlite3.connect("artifacts.db")
        cur = conn.cursor()

        # Drop + create tables
        cur.execute("DROP TABLE IF EXISTS artifact_metadata")
        cur.execute("DROP TABLE IF EXISTS artifact_media")
        cur.execute("DROP TABLE IF EXISTS artifact_colors")

        cur.execute(\"\"\"CREATE TABLE artifact_metadata (
            id INTEGER, title TEXT, culture TEXT, period TEXT, century TEXT,
            medium TEXT, dimensions TEXT, description TEXT,
            department TEXT, classification TEXT, accessionyear TEXT, accessionmethod TEXT
        )\"\"\")

        cur.execute(\"\"\"CREATE TABLE artifact_media (
            objectid INTEGER, imagecount INTEGER, mediacount INTEGER,
            colorcount INTEGER, rank INTEGER, datebegin INTEGER, dateend INTEGER
        )\"\"\")

        cur.execute(\"\"\"CREATE TABLE artifact_colors (
            objectid INTEGER, color TEXT, spectrum TEXT, hue TEXT, percent REAL, css3 TEXT
        )\"\"\")

        metadata, media, colors = [], [], []
        for rec in st.session_state.records:
            metadata.append((
                rec.get("id"), rec.get("title"), rec.get("culture"), rec.get("period"),
                rec.get("century"), rec.get("medium"), rec.get("dimensions"),
                rec.get("description"), rec.get("department"),
                rec.get("classification"), rec.get("accessionyear"), rec.get("accessionmethod")
            ))
            media.append((
                rec.get("objectid"), rec.get("imagecount"), rec.get("mediacount"),
                rec.get("colorcount"), rec.get("rank"),
                rec.get("datebegin"), rec.get("dateend")
            ))
            if rec.get("colors"):
                for j in rec["colors"]:
                    colors.append((
                        rec.get("objectid"), j.get("color"), j.get("spectrum"),
                        j.get("hue"), j.get("percent"), j.get("css3")
                    ))

        cur.executemany("INSERT INTO artifact_metadata VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", metadata)
        cur.executemany("INSERT INTO artifact_media VALUES (?,?,?,?,?,?,?)", media)
        cur.executemany("INSERT INTO artifact_colors VALUES (?,?,?,?,?,?)", colors)

        conn.commit()
        conn.close()
        st.session_state.db_ready = True
        st.success("✅ Data inserted into SQLite database!")

        # Preview
        st.subheader("📑Artifact Metadata")
        st.dataframe(pd.DataFrame(metadata[:100], columns=[
            "id","title","culture","period","century","medium","dimensions","description",
            "department","classification","accessionyear","accessionmethod"
        ]))

        st.subheader("🖼Artifact Media")
        st.dataframe(pd.DataFrame(media[:100], columns=[
            "objectid","imagecount","mediacount","colorcount","rank","datebegin","dateend"
        ]))

        st.subheader("🎨Artifact Colors")
        if colors:
            st.dataframe(pd.DataFrame(colors[:100], columns=[
                "objectid","color","spectrum","hue","percent","css3"
            ]))
        else:
            st.info("No colors found.")

# --- Step 3c: SQL Queries ---
if st.session_state.step == "sql" and st.session_state.db_ready:
    conn = sqlite3.connect("artifacts.db")

    st.markdown("## 🏺 SQL Queries")

    tab1, tab2, tab3, tab4, tab5 = st.tabs(["Metadata", "Media", "Colors", "Joins", "Own Queries"])

    # Metadata Queries
    with tab1:
        query_options = {
            "1. List all artifacts from the 11th century belonging to Byzantine culture.": \"""
                SELECT * FROM artifact_metadata
                WHERE century LIKE '%11th%' AND culture LIKE '%Byzantine%'
            \""",
            "2. What are the unique cultures represented in the artifacts?": \"""
                SELECT DISTINCT culture FROM artifact_metadata
                WHERE culture IS NOT NULL
            \""",
            "3. List all artifacts from the Archaic Period.": \"""
                SELECT * FROM artifact_metadata
                WHERE period LIKE '%Archaic%'
            \""",
            "4. List artifact titles ordered by accession year in descending order.": \"""
                SELECT title, accessionyear FROM artifact_metadata
                WHERE accessionyear IS NOT NULL
                ORDER BY accessionyear DESC
            \""",
            "5. How many artifacts are there per department?": \"""
                SELECT department, COUNT(*) as artifact_count
                FROM artifact_metadata
                GROUP BY department
                ORDER BY artifact_count DESC
            \"""
        }
        selected_query = st.selectbox("Select a metadata query:", list(query_options.keys()))
        if selected_query:
            df = pd.read_sql_query(query_options[selected_query], conn)
            st.dataframe(df)

    # Media Queries
    with tab2:
        query_options = {
            "6. Which artifacts have more than 1 image?": \"""
                SELECT * FROM artifact_media
                WHERE imagecount > 1
            \""",
            "7. What is the average rank of all artifacts?": \"""
                SELECT AVG(rank) as avg_rank FROM artifact_media
            \""",
            "8. Which artifacts have a higher colorcount than mediacount?": \"""
                SELECT * FROM artifact_media
                WHERE colorcount > mediacount
            \""",
            "9. List all artifacts created between 1500 and 1600.": \"""
                SELECT * FROM artifact_media
                WHERE datebegin >= 1500 AND dateend <= 1600
            \""",
            "10. How many artifacts have no media files?": \"""
                SELECT COUNT(*) as no_media FROM artifact_media
                WHERE mediacount = 0
            \"""
        }
        selected_query = st.selectbox("Select a media query:", list(query_options.keys()))
        if selected_query:
            df = pd.read_sql_query(query_options[selected_query], conn)
            st.dataframe(df)

    # Colors Queries
    with tab3:
        query_options = {
            "11. What are all the distinct hues used in the dataset?": \"""
                SELECT DISTINCT hue FROM artifact_colors
                WHERE hue IS NOT NULL
            \""",
            "12. What are the top 5 most used colors by frequency?": \"""
                SELECT color, COUNT(*) as freq
                FROM artifact_colors
                GROUP BY color
                ORDER BY freq DESC
                LIMIT 5
            \""",
            "13. What is the average coverage percentage for each hue?": \"""
                SELECT hue, AVG(percent) as avg_percent
                FROM artifact_colors
                GROUP BY hue
            \""",
            "14. List all colors used for a given artifact ID.": None,
            "15. What is the total number of color entries in the dataset?": \"""
                SELECT COUNT(*) as total_colors FROM artifact_colors
            \"""
        }
        selected_query = st.selectbox("Select a colors query:", list(query_options.keys()))
        if selected_query == "14. List all colors used for a given artifact ID.":
            artifact_id = st.text_input("Enter Artifact ID:")
            if artifact_id:
                sql = f"SELECT * FROM artifact_colors WHERE objectid = {artifact_id}"
                df = pd.read_sql_query(sql, conn)
                st.dataframe(df)
        elif selected_query:
            sql = query_options[selected_query]
            if sql:
                df = pd.read_sql_query(sql, conn)
                st.dataframe(df)

    # Join Queries
    with tab4:
        query_options = {
            "16. List artifact titles and hues for all artifacts belonging to the Byzantine culture.": \"""
                SELECT m.title, c.hue
                FROM artifact_metadata m
                JOIN artifact_colors c ON m.id = c.objectid
                WHERE m.culture LIKE '%Byzantine%'
            \""",
            "17. List each artifact title with its associated hues.": \"""
                SELECT m.title, c.hue
                FROM artifact_metadata m
                JOIN artifact_colors c ON m.id = c.objectid
            \""",
            "18. Get artifact titles, cultures, and media ranks where the period is not null.": \"""
                SELECT m.title, m.culture, media.rank
                FROM artifact_metadata m
                JOIN artifact_media media ON m.id = media.objectid
                WHERE m.period IS NOT NULL
            \""",
            "19. Find artifact titles ranked in the top 10 that include the color hue 'Grey'.": \"""
                SELECT m.title, media.rank, c.hue
                FROM artifact_metadata m
                JOIN artifact_media media ON m.id = media.objectid
                JOIN artifact_colors c ON m.id = c.objectid
                WHERE c.hue LIKE '%Grey%' AND media.rank <= 10
            \""",
            "20. How many artifacts exist per classification, and what is the average media count for each?": \"""
                SELECT m.classification, COUNT(*) as total_artifacts, AVG(media.mediacount) as avg_media
                FROM artifact_metadata m
                JOIN artifact_media media ON m.id = media.objectid
                GROUP BY m.classification
            \"""
        }
        selected_query = st.selectbox("Select a join-based query:", list(query_options.keys()))
        if selected_query:
            df = pd.read_sql_query(query_options[selected_query], conn)
            st.dataframe(df)

    # Own Queries (21–29)
    with tab5:
        query_options = {
            "21. List all the accession methods from 19th century (metadata).": \"""
                SELECT accessionmethod FROM artifact_metadata
                WHERE century LIKE '%19th%'
            \""",
            "22. List all titles from 20th century (metadata).": \"""
                SELECT title FROM artifact_metadata
                WHERE century LIKE '%20th%'
            \""",
            "23. List all artifacts where accessionyear between 1990 and 2000(metadata).": \"""
                SELECT * FROM artifact_metadata
                WHERE accessionyear BETWEEN 1990 AND 2000
            \""",
            "24. List all artifacts where datebegin and dateend are same(media).": \"""
                SELECT * FROM artifact_media
                WHERE datebegin = dateend
            \""",
            "25. List all ranks where imagecount, mediacount, and colorcount = 0 (media).": \"""
                SELECT rank, imagecount, mediacount, colorcount  FROM artifact_media
                WHERE imagecount = 0 AND mediacount = 0 AND colorcount = 0
            \""",
            "26. List all objectid, colorcount where colorcount greater than 5 (media).": \"""
                SELECT objectid, colorcount FROM artifact_media
                WHERE colorcount > 5
            \""",
            "27. List all distinct spectrums (colors).": \"""
                SELECT DISTINCT spectrum FROM artifact_colors
                WHERE spectrum IS NOT NULL
            \""",
            "28. List all IDs where hue = 'Brown' (colors).": \"""
                SELECT objectid FROM artifact_colors
                WHERE hue = 'Brown'
            \""",
            "29. List all percent values where hue = 'Green' (colors).": \"""
                SELECT percent FROM artifact_colors
                WHERE hue = 'Green'
            \""",
            "30. list all artifact where unique culture and color count =7 , hue = brown.": \"""
                 SELECT DISTINCT m.id, m.title, m.culture, media.colorcount, c.hue
                 FROM artifact_metadata m
                 JOIN artifact_media media ON m.id = media.objectid
                 JOIN artifact_colors c ON m.id = c.objectid
                 WHERE media.colorcount = 7
                 AND c.hue = 'Brown';
            \"""
        }
        selected_query = st.selectbox("Select an own query:", list(query_options.keys()))
        if selected_query:
            sql = query_options[selected_query]
            df = pd.read_sql_query(sql, conn)
            st.dataframe(df)

    conn.close()
"""

with open("app.py", "w") as f:
    f.write(app_code)

# --- 3. Run Streamlit in background ---
!nohup streamlit run app.py --server.port 8501 --server.headless true > log.txt 2>&1 &

# --- 4. Start tunnel with cloudflared ---
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate

(Reading database ... 126378 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.8.1) over (2025.8.1) ...
Setting up cloudflared (2025.8.1) ...
Processing triggers for man-db (2.10.2-1) ...
2025-09-06T14:13:10Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-06T14:13:10Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-09-06T14:13:13Z INF +---------------------------------